In [2]:
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/competitions/cpe-ai-hackathon-2025-medical-image-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: rokabpy
Your Kaggle Key: ··········


100%|██████████| 4.03G/4.03G [00:48<00:00, 89.6MB/s]



Extracting archive ./cpe-ai-hackathon-2025-medical-image-classification/cpe-ai-hackathon-2025-medical-image-classification.zip to ./cpe-ai-hackathon-2025-medical-image-classification


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
train = pd.read_csv("/content/cpe-ai-hackathon-2025-medical-image-classification/liver-fibrosis-severity-prediction/train.csv")
final = pd.read_csv("/content/cpe-ai-hackathon-2025-medical-image-classification/liver-fibrosis-severity-prediction/test.csv")
train = train.drop(columns=['subject'])
train = train.rename(columns={
    'image_name' : 'img',
    'SWE fibrosis stage' : 'stage',
    'TE result' : 'TE'
})

id_final = final.ID
final = final.drop(columns=['ID'])
final = final.rename(columns={
    'image_id' : 'img',
    'SWE fibrosis stage' : 'stage',
    'TE result' : 'TE'
})

In [5]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   img     334 non-null    object
 1   view    334 non-null    object
 2   stage   334 non-null    object
 3   TE      334 non-null    object
dtypes: object(4)
memory usage: 10.6+ KB


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1438 entries, 0 to 1437
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   img      1438 non-null   object 
 1   view     1438 non-null   object 
 2   stage    1438 non-null   object 
 3   TE(kPa)  1438 non-null   float64
 4   TE       1438 non-null   object 
dtypes: float64(1), object(4)
memory usage: 56.3+ KB


In [7]:
np.unique(train['view'])

array(['Intercostal', 'Liver/RK', 'Subcostal_hepatic_vein'], dtype=object)

In [8]:
np.unique(train['stage'])

array(['-', 'F0-1', 'F2', 'F2 ', 'F3', 'F4'], dtype=object)

In [9]:
np.unique(train['TE'])

array(['F0', 'F1', 'F2', 'F3', 'F4'], dtype=object)

In [10]:
train = train.drop(columns='img')

In [11]:
y_train = train['TE(kPa)']
x_train = train.drop(columns='TE(kPa)')

In [12]:
x_train

,view,stage,TE
0,Intercostal,-,F2
1,Liver/RK,F0-1,F0
2,Subcostal_hepatic_vein,F0-1,F0
3,Liver/RK,F0-1,F0
4,Intercostal,F0-1,F1
...,...,...,...
1433,Intercostal,F0-1,F0
1434,Liver/RK,F0-1,F0
1435,Liver/RK,F0-1,F1
1436,Intercostal,F0-1,F1


In [13]:
final = final.drop(columns='img')
final

,view,stage,TE
0,Intercostal,-,F0
1,Subcostal_hepatic_vein,F2,F3
2,Intercostal,F2,F3
3,Liver/RK,F2,F3
4,Subcostal_hepatic_vein,F0-1,F0
...,...,...,...
329,Intercostal,F0-1,F4
330,Subcostal_hepatic_vein,F0-1,F4
331,Subcostal_hepatic_vein,F0-1,F0
332,Intercostal,F0-1,F0


In [14]:
x_train['stage'] = x_train['stage'].str.strip().map({
    '-' : 0,
    'F0-1' : 1,
    'F2' : 2,
    'F3' : 3,
    'F4' : 4
})

x_train['TE'] = x_train['TE'].map({
    'F0' : 0,
    'F1' : 1,
    'F2' : 2,
    'F3' : 3,
    'F4' : 4
})

x_train = pd.get_dummies(x_train, columns=['view'], dtype=int)

final['stage'] = final['stage'].str.strip().map({
    '-' : 0,
    'F0-1' : 1,
    'F2' : 2,
    'F3' : 3,
    'F4' : 4
})

final['TE'] = final['TE'].map({
    'F0' : 0,
    'F1' : 1,
    'F2' : 2,
    'F3' : 3,
    'F4' : 4
})

final = pd.get_dummies(final, columns=['view'], dtype=int)

In [15]:
x_train = x_train.values.astype(np.float32)
y_train = y_train.values.astype(np.float32).reshape(-1, 1)
tensor_x_train = torch.tensor(x_train)
tensor_y_train = torch.tensor(y_train)

final = final.values.astype(np.float32)
tensor_final = torch.tensor(final)

In [18]:
class neural_network(nn.Module):
    def __init__(self, input_dim):
        super(neural_network, self).__init__()
        self.feature = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.3),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.3)
        )
        self.input_bn = nn.BatchNorm1d(input_dim)
        self.output_layer = nn.Linear(32, 1)

    def forward(self, x):
        x = self.input_bn(x)
        x = self.feature(x)
        out = self.output_layer(x)
        return out

In [19]:
model = neural_network(x_train.shape[1])
error = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

epoch, n = 3000, 100
for i in range(epoch):
    model.train()
    y_hat = model(tensor_x_train)
    loss = error(y_hat, tensor_y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if i == (n-1):
        print(f"Epoch [{i+1}/{epoch}] | Loss(MSE): {loss.item():.4f}")
        n += 100

Epoch [100/3000] | Loss(MSE): 4.7998
Epoch [200/3000] | Loss(MSE): 4.1611
Epoch [300/3000] | Loss(MSE): 4.3769
Epoch [400/3000] | Loss(MSE): 4.6129
Epoch [500/3000] | Loss(MSE): 4.3403
Epoch [600/3000] | Loss(MSE): 3.9471
Epoch [700/3000] | Loss(MSE): 4.0799
Epoch [800/3000] | Loss(MSE): 3.8984
Epoch [900/3000] | Loss(MSE): 3.8442
Epoch [1000/3000] | Loss(MSE): 4.3705
Epoch [1100/3000] | Loss(MSE): 3.9723
Epoch [1200/3000] | Loss(MSE): 3.9747
Epoch [1300/3000] | Loss(MSE): 4.0062
Epoch [1400/3000] | Loss(MSE): 3.9504
Epoch [1500/3000] | Loss(MSE): 3.7234
Epoch [1600/3000] | Loss(MSE): 3.6553
Epoch [1700/3000] | Loss(MSE): 3.9996
Epoch [1800/3000] | Loss(MSE): 4.0601
Epoch [1900/3000] | Loss(MSE): 3.7797
Epoch [2000/3000] | Loss(MSE): 3.8139
Epoch [2100/3000] | Loss(MSE): 3.8743
Epoch [2200/3000] | Loss(MSE): 3.8882
Epoch [2300/3000] | Loss(MSE): 3.6350
Epoch [2400/3000] | Loss(MSE): 3.9424
Epoch [2500/3000] | Loss(MSE): 3.6772
Epoch [2600/3000] | Loss(MSE): 3.7861
Epoch [2700/3000] | L

In [20]:
with torch.no_grad():
  y_hat = model(tensor_final)
y_hat = pd.DataFrame(y_hat)
summarize = pd.concat([id_final, y_hat], axis=1)
summarize.columns = ['ID', 'TE(kPa)']
summarize.to_csv('My_Answer.csv', index=False)
summarize

,ID,TE(kPa)
0,1,4.378418
1,2,9.416288
2,3,9.982777
3,4,9.119219
4,5,4.417382
...,...,...
329,330,14.975607
330,331,12.497107
331,332,4.501649
332,333,4.381536
